In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

df = pd.read_excel('dm\hw2.xls')

row=df.shape[0]#取10~12月當資料
date=row-(92*18)
df=df.iloc[date:row]

df=df.replace({'NR': 0})#NR表示無降雨，以0取代

#清理資料，將空白以及缺失、無效值取平均
for i in range(92*18):
    for j in range(3,27):
        x=df.iloc[i,j]
        if(x+x!=2*x or type(x)==str):
            findrear=0
            findfront=0
            for a in range(j+1,27):
                rear=df.iloc[i,a]
                if(type(rear)!=str):
                 findrear=1
                 break        
            b=j-1
            while b > 2:
                front=df.iloc[i,b]
                if(type(front)!=str):
                 findfront=1
                 break
                b=b-1
            if(rear+rear!=2*rear):
                rear=front
            if(front+front!=2*front):
                front=rear  
            if(type(rear)==str):
                rear=front
            if(type(front)==str):
                front=rear  
            df.iloc[i,j]=(rear+front)/2
            final=df.iloc[i,j]
                
                
                
                
#切割訓練及測試集              
row=df.shape[0]                
test_set=row-(31*18)
train=df.iloc[0:test_set]
test=df.iloc[test_set:row]
train=train.drop(['日期','測站','測項'], axis=1)
test=test.drop(['日期','測站','測項'], axis=1)
train_set=train.iloc[0:18]

#train的時序資料
train_set.index = range(1,len(train_set) + 1)
i=18
while i<61*18:
    aa=train.iloc[i:i+18]
    aa.index = range(1,len(aa) + 1)
    train_set=pd.concat([train_set,aa],axis=1)
    i=i+18

#test的時序資料   
test_set=test.iloc[0:18]
test_set.index = range(1,len(train_set) + 1)
j=18
while j<31*18:
    aa=test.iloc[j:j+18]
    aa.index = range(1,len(aa) + 1)
    test_set=pd.concat([test_set,aa],axis=1)
    j=j+18
    
#製作train的時間序列
train_list = []#18種資料
train_list2 = []#只有PM2.5
train_listy = []#正確的y值
i=0
j=6
while i<61*24-6:
    a_set=train_set.iloc[0:18,i:i+6].values
    b_set=train_set.iloc[9,i:i+6].values
    A = np.asarray(a_set).ravel()
    B = np.asarray(b_set).ravel()
    train_list.append(A)
    train_list2.append(B)
    i=i+1
while j<61*24:
    zxczxc=train_set.iloc[9,j]
    train_listy.append(zxczxc)
    j=j+1

#製作test的時間序列
test_list = []#18種資料
test_list2 = []#只有PM2.5
test_listy = []#正確的y值    
i=0
j=6
while i<31*24-6:
    a_set=test_set.iloc[0:18,i:i+6].values
    b_set=test_set.iloc[9,i:i+6].values
    A = np.asarray(a_set).ravel()
    B = np.asarray(b_set).ravel()
    test_list.append(A)
    test_list2.append(B)
    i=i+1
while j<31*24:
    zxczxc=test_set.iloc[9,j]
    test_listy.append(zxczxc)
    j=j+1

#套Linear Regression 
reg = LinearRegression().fit(train_list, train_listy)
reg2 = LinearRegression().fit(train_list2, train_listy)
predict = reg.predict(test_list)
predict2 = reg2.predict(test_list2)
mean = mean_absolute_error(test_listy, predict)
mean2 = mean_absolute_error(test_listy, predict2)

#套Random Forest Regression
regr = RandomForestRegressor(max_depth=10, random_state=0,n_estimators=100,criterion='mae',max_features = 12)
regr2 = RandomForestRegressor(max_depth=10, random_state=0,n_estimators=100,criterion='mae')
regr.fit(train_list, train_listy)
regr2.fit(train_list2, train_listy)
predict3 = regr.predict(test_list)
predict4 = regr2.predict(test_list2)
mean3 = mean_absolute_error(test_listy, predict3)
mean4 = mean_absolute_error(test_listy, predict4)

print('18種資料的Linear Regression')
print('score:',reg.score(train_list, train_listy))
print(mean)
print(                              )

print('只有PM2.5的Linear Regression')
print('score:',reg2.score(train_list2, train_listy))
print('MAE:',mean2)
print(                              )

print('18種資料的Random Forest Regression')
print('score:',regr.score(train_list,train_listy))
print('MAE:',mean3)
print(                              )

print('只有PM2.5的Random Forest Regression')
print('score:',regr2.score(train_list2,train_listy))
print('MAE:',mean4)


18種資料的Linear Regression
score: 0.9266971820201474
2.8622061210704146

只有PM2.5的Linear Regression
score: 0.9032239905016829
MAE: 2.5694851550301685

18種資料的Random Forest Regression
score: 0.9584138049098401
MAE: 3.755616531165311

只有PM2.5的Random Forest Regression
score: 0.9554511542595636
MAE: 3.0316056910569102
